In [ ]:
!pip install fsspec==2024.10.0


In [ ]:
!pip install gcsfs --upgrade


In [ ]:
!pip install --upgrade --force-reinstall fsspec gcsfs

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached gcsfs-2024.10.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached aiohttp-3.11.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached google_auth-2.37.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux201

In [ ]:
import fsspec
import gcsfs

print(f"fsspec version: {fsspec.__version__}")
print(f"gcsfs version: {gcsfs.__version__}")


fsspec version: 2024.10.0
gcsfs version: 2024.10.0


In [7]:
# Install required libraries
!pip install transformers datasets -q

In [8]:
# Import necessary libraries
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [14]:
# Step 1: Load Dataset
# Use the Sentiment140 dataset or any available dataset from Hugging Face datasets
dataset = load_dataset("sentiment140")

In [15]:
# Split into train and test
train_data = dataset['train'].shuffle(seed=42).select(range(20000))  # Train on 20,000 samples
test_data = dataset['test'].shuffle(seed=42).select(range(498))     # Use all 498 samples for testing

In [16]:
print(f"Train size: {len(dataset['train'])}")
print(f"Test size: {len(dataset['test'])}")

Train size: 1600000
Test size: 498


In [17]:
# Step 2: Preprocess Tweets
def preprocess_tweet(tweet):
    import re
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)  # Remove special characters
    tweet = tweet.lower().strip()  # Convert to lowercase and strip whitespaces
    return tweet

# Apply preprocessing
train_data = train_data.map(lambda x: {'text': preprocess_tweet(x['text'])})
test_data = test_data.map(lambda x: {'text': preprocess_tweet(x['text'])})

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [18]:
# Step 3: Tokenize the Dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

# Remove unnecessary columns for training
train_dataset = train_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])

# Set format for PyTorch tensors
train_dataset.set_format("torch")
test_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [19]:
# Step 4: Load Pretrained Model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Step 5: Define Evaluation Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [21]:
# Step 6: Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# Step 7: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-22-7eb1e0e6bbd4>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding=True)
    tokenized["labels"] = examples["label"]  # Add labels from the dataset
    return tokenized


In [28]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 1600000
    })
    test: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 498
    })
})
{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 'date': 'Mon Apr 06 22:19:45 PDT 2009', 'user': '_TheSpecialOne_', 'sentiment': 0, 'query': 'NO_QUERY'}


In [29]:
def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding=True)
    tokenized["labels"] = examples["sentiment"]  # Replace 'sentiment' with the correct key if different
    return tokenized


In [30]:
def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding=True)
    # Map labels: 0 -> 0 (negative), 4 -> 1 (positive)
    tokenized["labels"] = [0 if label == 0 else 1 for label in examples["sentiment"]]
    return tokenized


In [31]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [32]:
print(tokenized_datasets["train"][0])

{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 'date': 'Mon Apr 06 22:19:45 PDT 2009', 'user': '_TheSpecialOne_', 'sentiment': 0, 'query': 'NO_QUERY', 'input_ids': [101, 1030, 6942, 13064, 8299, 1024, 1013, 1013, 1056, 9148, 25856, 2594, 1012, 4012, 1013, 1016, 2100, 2487, 2480, 2140, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


In [34]:
dataset = load_dataset("imdb")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [35]:
print(tokenized_datasets["train"][0])  # Inspect a sample from the train dataset


{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 'date': 'Mon Apr 06 22:19:45 PDT 2009', 'user': '_TheSpecialOne_', 'sentiment': 0, 'query': 'NO_QUERY', 'input_ids': [101, 1030, 6942, 13064, 8299, 1024, 1013, 1013, 1056, 9148, 25856, 2594, 1012, 4012, 1013, 1016, 2100, 2487, 2480, 2140, 1011, 22091, 2860, 2860, 1010, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1025, 1040, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


In [40]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",  # Directory to save model checkpoints
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save checkpoints at the end of each epoch
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=8,  # Adjust batch size based on GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=1,  # Number of training epochs
    weight_decay=0.01,
    logging_dir="./logs",  # Directory for logs
    logging_steps=10,  # Log every 10 steps
    report_to="none",  # Disable Weights & Biases
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,  # Ensures correct tokenization
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-40-6e1407ca211a>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
print(len(tokenized_datasets["train"]))  # Size of train dataset
print(len(tokenized_datasets["test"]))   # Size of test dataset



1600000
498


In [44]:
train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))  # Use 10,000 samples for training
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(498))     # Use all available test samples


In [45]:
train_data = tokenized_datasets["train"]
test_data = tokenized_datasets["test"]


In [46]:
train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(min(10000, len(tokenized_datasets["train"]))))
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(min(2000, len(tokenized_datasets["test"]))))


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)

trainer.train()


<ipython-input-47-05987fffada1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.419800,0.385601


TrainOutput(global_step=1250, training_loss=0.37514550285339354, metrics={'train_runtime': 135.7346, 'train_samples_per_second': 73.673, 'train_steps_per_second': 9.209, 'total_flos': 292683657622464.0, 'train_loss': 0.37514550285339354, 'epoch': 1.0})

In [48]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",       # Directory to save model checkpoints
    evaluation_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch",       # Save model after each epoch
    learning_rate=2e-5,          # Learning rate
    per_device_train_batch_size=8,  # Adjust batch size for your GPU
    per_device_eval_batch_size=8,
    num_train_epochs=3,          # Number of training epochs
    weight_decay=0.01,           # Regularization
    logging_dir="./logs",        # Directory for logs
    logging_steps=10,            # Log every 10 steps
    report_to="none",            # Disable Weights & Biases
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-48-566d053db483>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [49]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.497800,0.413918
2,0.221300,0.766284
3,0.211600,0.949412


TrainOutput(global_step=3750, training_loss=0.1951303392936786, metrics={'train_runtime': 412.8861, 'train_samples_per_second': 72.659, 'train_steps_per_second': 9.082, 'total_flos': 878181372788400.0, 'train_loss': 0.1951303392936786, 'epoch': 3.0})

In [50]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.949411928653717, 'eval_runtime': 0.9478, 'eval_samples_per_second': 525.426, 'eval_steps_per_second': 66.47, 'epoch': 3.0}


In [61]:
import torch

def predict_sentiment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Ensure the model is on the correct device

    # Tokenize and move input tensors to the same device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return "Positive" if predictions.item() == 1 else "Negative"

# Example usage
print(predict_sentiment("I am unlucky"))
print(predict_sentiment("I failed in exam"))



Negative
Negative
